In [1]:
""" This script tries only using the TuLiP toolbox to solve a multirobot motion planning problem.
Specifically, a field of cells and a number of robots are specified along with the system dynamics,
and constraints are written in monolithic LTL syntax.
The difference between this script and TuLiP_multirobot_solver.py
"""

# Import the packages that we need
from __future__ import print_function
import numpy as np
from tulip import spec
from tulip.transys import machines
from tulip import synth
import gr1_fragment
import State_encoding
import LTL_encoding

`omega.symbolic.symbolic` failed to import `dd.cudd`.
Will use `dd.autoref`.


In [2]:
# Workspace parameters
num_rows = 3
num_cols = 2
num_robots = 2
delay = 1
robot_init_pos = np.array([[0, 1], [1, 0]])
# The environment variables
env_vars = {}
env_init = set()
env_prog = set()
env_safe = set()

In [3]:
for r in range(1, num_robots + 1):
    env_vars[('x' + str(r))] = (0, num_cols)
    env_vars[('y' + str(r))] = (0, num_rows)
    env_vars[('c' + str(r))] = (-1, delay + 1)
    env_vars[('delay' + str(r))] = 'boolean'
print(env_vars)

{'x1': (0, 2), 'y1': (0, 3), 'c1': (-1, 2), 'delay1': 'boolean', 'x2': (0, 2), 'y2': (0, 3), 'c2': (-1, 2), 'delay2': 'boolean'}


In [4]:
env_init |= {'c1=0'}
env_init |= {'c2=0'}
for r in range(1, num_robots + 1):
    env_init |= {'x' + str(r) + '=' + str(robot_init_pos[r - 1][0])}
    env_init |= {'y' + str(r) + '=' + str(robot_init_pos[r - 1][1])}
print(env_init)

{'c2=0', 'y1=1', 'c1=0', 'x2=1', 'x1=0', 'y2=0'}


In [5]:
# Now no need to encode that each robot should only appear in one position
# Encode the transition relations
env_trans_x = set()
env_trans_y = set()
for r in range(1, num_robots + 1):
    for col in range(num_cols):
        if col != num_cols - 1:
            env_trans_x |= {'( x' + str(r) + '=' + str(col) + ' && right' + str(r) + ' ) -> ( ( X(x' + str(r) + '=' +
                            str(col+1) + ') && !delay' + str(r) + ' ) || ( X(x' + str(r) + '=' + str(col) +
                            ') && delay' + str(r) + ' ) )'}
        if col != 0:
            env_trans_x |= {'( x' + str(r) + '=' + str(col) + ' && left' + str(r) + ' ) -> ( ( X(x' + str(r) + '=' +
                            str(col - 1) + ') && !delay' + str(r) + ' ) || ( X(x' + str(r) + '=' + str(col) +
                            ') && delay' + str(r) + ' ) )'}
        env_trans_x |= {'( x' + str(r) + '=' + str(col) + ' && up' + str(r) + ' ) -> X( x' + str(r) + '=' + str(col) +
                        ' )'}
        env_trans_x |= {'( x' + str(r) + '=' + str(col) + ' && down' + str(r) + ' ) -> X( x' + str(r) + '=' + str(col) +
                        ' )'}
        env_trans_x |= {'( x' + str(r) + '=' + str(col) + ' && stop' + str(r) + ' ) -> ( X( x' + str(r) + '=' + str(col) +
                        ' ) && ( !delay' + str(r) + ' ) )'}
    for row in range(num_rows):
        if row != num_rows - 1:
            env_trans_y |= {'( y' + str(r) + '=' + str(row) + ' && up' + str(r) + ' ) -> ( ( X(y' + str(r) + '=' +
                            str(row + 1) + ') && !delay' + str(r) + ' ) || ( X(y' + str(r) + '=' + str(row) +
                            ') && delay' + str(r) + ' ) )'}
        if row != 0:
            env_trans_y |= {'( y' + str(r) + '=' + str(row) + ' && down' + str(r) + ' ) -> ( ( X(y' + str(r) + '=' +
                            str(row - 1) + ') && !delay' + str(r) + ' ) || ( X(y' + str(r) + '=' + str(row) +
                            ') && delay' + str(r) + ' ) )'}
        env_trans_y |= {'( y' + str(r) + '=' + str(row) + ' && left' + str(r) + ' ) -> X( y' + str(r) + '=' + str(row) +
                        ' )'}
        env_trans_y |= {'( y' + str(r) + '=' + str(row) + ' && right' + str(r) + ' ) -> X( y' + str(r) + '=' + str(row) +
                        ' )'}
        env_trans_y |= {'( y' + str(r) + '=' + str(row) + ' && stop' + str(r) + ' ) -> ( X( y' + str(r) + '=' + str(row) +
                        ' ) && ( !delay' + str(r) + ' ) )'}
print(env_trans_x)
print(env_trans_y)
env_safe |= env_trans_x
env_safe |= env_trans_y

{'( x1=0 && down1 ) -> X( x1=0 )', '( x1=0 && up1 ) -> X( x1=0 )', '( x1=1 && left1 ) -> ( ( X(x1=0) && !delay1 ) || ( X(x1=1) && delay1 ) )', '( x1=1 && stop1 ) -> ( X( x1=1 ) && ( !delay1 ) )', '( x2=0 && up2 ) -> X( x2=0 )', '( x2=0 && stop2 ) -> ( X( x2=0 ) && ( !delay2 ) )', '( x2=1 && down2 ) -> X( x2=1 )', '( x2=0 && down2 ) -> X( x2=0 )', '( x1=0 && stop1 ) -> ( X( x1=0 ) && ( !delay1 ) )', '( x1=1 && down1 ) -> X( x1=1 )', '( x2=0 && right2 ) -> ( ( X(x2=1) && !delay2 ) || ( X(x2=0) && delay2 ) )', '( x2=1 && stop2 ) -> ( X( x2=1 ) && ( !delay2 ) )', '( x2=1 && left2 ) -> ( ( X(x2=0) && !delay2 ) || ( X(x2=1) && delay2 ) )', '( x2=1 && up2 ) -> X( x2=1 )', '( x1=1 && up1 ) -> X( x1=1 )', '( x1=0 && right1 ) -> ( ( X(x1=1) && !delay1 ) || ( X(x1=0) && delay1 ) )'}
{'( y2=2 && down2 ) -> ( ( X(y2=1) && !delay2 ) || ( X(y2=2) && delay2 ) )', '( y2=1 && down2 ) -> ( ( X(y2=0) && !delay2 ) || ( X(y2=1) && delay2 ) )', '( y1=2 && left1 ) -> X( y1=2 )', '( y1=1 && left1 ) -> X( y1=1 

In [6]:
# Relate the delay variables (delay#) to the clock variables (c#)
delay2clock_minus = '( '
delay2clock_plus = ''
for r in range(1, num_robots + 1):
    delay2clock_minus += '( delay' + str(r) + ' && c' + str(r) + '=' + str(delay) + ' ) || '
delay2clock_minus = delay2clock_minus[:-3] + ') -> ( '
delay2clock_plus = '!' + delay2clock_minus
for r in range(1, num_robots + 1):
    for c in range(delay + 1):
        delay2clock_minus += '( ( !delay' + str(r) + ' && c' + str(r) + '=' + str(c) + ' ) -> X(c' + str(r) + '=' + \
                             str(c - 1) + ') ) && '
        delay2clock_minus += '( ( delay' + str(r) + ' && c' + str(r) + '=' + str(c) + ' ) -> X(c' + str(r) + '=' + \
                             str(c) + ') ) && '
        delay2clock_plus += '( ( delay' + str(r) + ' && c' + str(r) + '=' + str(c) + ' ) -> X(c' + str(r) + '=' + \
                            str(c + 1) + ') ) && '
        delay2clock_plus += '( ( !delay' + str(r) + ' && c' + str(r) + '=' + str(c) + ' ) -> X(c' + str(r) + '=' + \
                            str(c) + ') ) && '
delay2clock_minus = delay2clock_minus[:-3] + ')'
delay2clock_plus = delay2clock_plus[:-3] + ')'
print(delay2clock_minus)
print(delay2clock_plus)
env_safe |= {delay2clock_minus}
env_safe |= {delay2clock_plus}

( ( delay1 && c1=1 ) || ( delay2 && c2=1 ) ) -> ( ( ( !delay1 && c1=0 ) -> X(c1=-1) ) && ( ( delay1 && c1=0 ) -> X(c1=0) ) && ( ( !delay1 && c1=1 ) -> X(c1=0) ) && ( ( delay1 && c1=1 ) -> X(c1=1) ) && ( ( !delay2 && c2=0 ) -> X(c2=-1) ) && ( ( delay2 && c2=0 ) -> X(c2=0) ) && ( ( !delay2 && c2=1 ) -> X(c2=0) ) && ( ( delay2 && c2=1 ) -> X(c2=1) ) )
!( ( delay1 && c1=1 ) || ( delay2 && c2=1 ) ) -> ( ( ( delay1 && c1=0 ) -> X(c1=1) ) && ( ( !delay1 && c1=0 ) -> X(c1=0) ) && ( ( delay1 && c1=1 ) -> X(c1=2) ) && ( ( !delay1 && c1=1 ) -> X(c1=1) ) && ( ( delay2 && c2=0 ) -> X(c2=1) ) && ( ( !delay2 && c2=0 ) -> X(c2=0) ) && ( ( delay2 && c2=1 ) -> X(c2=2) ) && ( ( !delay2 && c2=1 ) -> X(c2=1) ) )


In [7]:
# Define sys_vars
sys_vars = {}
for r in range(1, num_robots + 1):
    sys_vars['stop' + str(r)] = 'boolean'
    sys_vars['left' + str(r)] = 'boolean'
    sys_vars['right' + str(r)] = 'boolean'
    sys_vars['up' + str(r)] = 'boolean'
    sys_vars['down' + str(r)] = 'boolean'
print(sys_vars)

{'stop1': 'boolean', 'left1': 'boolean', 'right1': 'boolean', 'up1': 'boolean', 'down1': 'boolean', 'stop2': 'boolean', 'left2': 'boolean', 'right2': 'boolean', 'up2': 'boolean', 'down2': 'boolean'}


In [8]:
# Define initial command. Assume nothing.
sys_init = set()

sys_safe = set()
sys_prog = set()

In [9]:
# Constraint on the maximum delay
sys_max_delay = set()
for r in range(1, num_robots + 1):
    sys_max_delay |= {'!(c' + str(r) + '=' + '-1)'}
    sys_max_delay |= {'!(c' + str(r) + '=' + str(delay + 1) + ')'}
print(sys_max_delay)
sys_safe |= sys_max_delay

{'!(c1=2)', '!(c1=-1)', '!(c2=-1)', '!(c2=2)'}


In [10]:
# Avoid bad actions
sys_bad_act = set()
for r in range(1, num_robots + 1):
    sys_bad_act |= {'x' + str(r) + '=0 -> !left' + str(r)}
    sys_bad_act |= {'x' + str(r) + '=' + str(num_cols - 1) + ' -> !right' + str(r)}
    sys_bad_act |= {'y' + str(r) + '=0 -> !down' + str(r)}
    sys_bad_act |= {'y' + str(r) + '=' + str(num_rows - 1) + ' -> !up' + str(r)}
print(sys_bad_act)
sys_safe |= sys_bad_act

{'x1=1 -> !right1', 'y2=2 -> !up2', 'x2=1 -> !right2', 'y1=0 -> !down1', 'x1=0 -> !left1', 'y1=2 -> !up1', 'y2=0 -> !down2', 'x2=0 -> !left2'}


In [11]:
# Encode the constraint that each time exactly one action must be chosen
sys_good_act = set()
for r in range(1, num_robots + 1):
    sys_good_act |= {'( left' + str(r) + ' ) <-> ( ' + '!right' + str(r) + ' && ' + '!up' + str(r) + ' && ' + '!down' +
                str(r) + ' && ' + '!stop' + str(r) + ' )'}
    sys_good_act |= {'( right' + str(r) + ' ) <-> ( ' + '!left' + str(r) + ' && ' + '!up' + str(r) + ' && ' + '!down' +
                str(r) + ' && ' + '!stop' + str(r) + ' )'}
    sys_good_act |= {'( up' + str(r) + ' ) <-> ( ' + '!right' + str(r) + ' && ' + '!left' + str(r) + ' && ' + '!down' +
                str(r) + ' && ' + '!stop' + str(r) + ' )'}
    sys_good_act |= {'( down' + str(r) + ' ) <-> ( ' + '!right' + str(r) + ' && ' + '!up' + str(r) + ' && ' + '!left' +
                str(r) + ' && ' + '!stop' + str(r) + ' )'}
    sys_good_act |= {'( stop' + str(r) + ' ) <-> ( ' + '!right' + str(r) + ' && ' + '!up' + str(r) + ' && ' + '!down' +
                str(r) + ' && ' + '!left' + str(r) + ' )'}
sys_safe |= sys_good_act
print(sys_good_act)

{'( left2 ) <-> ( !right2 && !up2 && !down2 && !stop2 )', '( down2 ) <-> ( !right2 && !up2 && !left2 && !stop2 )', '( down1 ) <-> ( !right1 && !up1 && !left1 && !stop1 )', '( up1 ) <-> ( !right1 && !left1 && !down1 && !stop1 )', '( right1 ) <-> ( !left1 && !up1 && !down1 && !stop1 )', '( stop1 ) <-> ( !right1 && !up1 && !down1 && !left1 )', '( right2 ) <-> ( !left2 && !up2 && !down2 && !stop2 )', '( up2 ) <-> ( !right2 && !left2 && !down2 && !stop2 )', '( left1 ) <-> ( !right1 && !up1 && !down1 && !stop1 )', '( stop2 ) <-> ( !right2 && !up2 && !down2 && !left2 )'}


In [12]:
# Encode the constraint that if the last action was not executed yet, the same action should the next action
sys_same_act = set()
for r in range(1, num_robots + 1):
    sys_same_act |= {'( delay' + str(r) + ' && left' + str(r) + ' ) -> X( left' + str(r) + ' )'}
    sys_same_act |= {'( delay' + str(r) + ' && right' + str(r) + ' ) -> X( right' + str(r) + ' )'}
    sys_same_act |= {'( delay' + str(r) + ' && up' + str(r) + ' ) -> X( up' + str(r) + ' )'}
    sys_same_act |= {'( delay' + str(r) + ' && down' + str(r) + ' ) -> X( down' + str(r) + ' )'}
print(sys_same_act)
sys_safe |= sys_same_act

{'( delay2 && down2 ) -> X( down2 )', '( delay1 && up1 ) -> X( up1 )', '( delay1 && right1 ) -> X( right1 )', '( delay2 && up2 ) -> X( up2 )', '( delay1 && down1 ) -> X( down1 )', '( delay1 && left1 ) -> X( left1 )', '( delay2 && right2 ) -> X( right2 )', '( delay2 && left2 ) -> X( left2 )'}


In [13]:
# User-defined system requirements
# Constraint 1: []<>([[0, 0], [0, 1], [0, 2]], 2)
# Constraint 2: []!([1, 1], 1)
# Constraint 3: ([]<>[1, 2], 2)

sys_prog |= {'(x1=0) && (x2=0)'}
sys_safe |= {'!(x1=1 && y1=1) && !(x2=1 && y2=1)'}
sys_prog |= {'x1=1 && y1=2'}
sys_prog |= {'x2=1 && y2=2'}

In [14]:
# Create a GR(1) specification
specs = spec.GRSpec(env_vars, sys_vars, env_init, sys_init,
                    env_safe, sys_safe, env_prog, sys_prog)
specs.qinit = '\E \A'  # Moore initial condition synthesized too
specs.moore = False
specs.plus_one = False

In [15]:
specs.check_syntax()

In [ ]:
print('Start synthesis')
ctrl = synth.synthesize(specs)
print('End synthesis')
assert ctrl is not None, 'unrealizable'

Start synthesis


In [ ]:
machines.random_run(ctrl, N=30)